<a href="https://colab.research.google.com/github/MaSaKaIV/QAwithOwnLLM/blob/main/finetuned_stablelm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# パッケージのインストール
!pip install llama-index
!pip install transformers accelerate bitsandbytes
!pip install sentencepiece einops sentence_transformers
!pip install git+https://github.com/huggingface/peft.git
!pip install datasets
!pip install pypdf
!pip install trl

  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-ho0h4u6a
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-ho0h4u6a
  Resolved https://github.com/huggingface/peft.git to commit 85013987aa82aa1af3da1236b6902556ce3e483e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
# パッケージのインポート
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [3]:
# データセットの読み込み
dataset = load_dataset("bbz662bbz/databricks-dolly-15k-ja-gozarinnemon", split="train")

# プロンプトテンプレートの準備
def generate_prompt(data_point):
    if data_point["input"]:
        result = f"""[INST] {data_point["instruction"]}\n\n{data_point["input"]} [/INST] {data_point["output"]}"""
    else:
        result = f"""[INST] {data_point["instruction"]} [/INST] {data_point["output"]}"""
    return result

# テキスト列の追加
def add_text(example):
    example["text"] = generate_prompt(example)
    del example["index"]
    del example["category"]
    del example["instruction"]
    del example["input"]
    del example["output"]
    return example
dataset = dataset.map(add_text)
print(dataset)
print(dataset[0]["text"])

Dataset({
    features: ['text'],
    num_rows: 15015
})
[INST] ヴァージン・オーストラリア航空はいつから運航を開始したのですか？

ヴァージン・オーストラリア航空（Virgin Australia Airlines Pty Ltd）はオーストラリアを拠点とするヴァージン・ブランドを冠する最大の船団規模を持つ航空会社です。2000年8月31日に、ヴァージン・ブルー空港として、2機の航空機、1つの空路を運行してサービスを開始しました。2001年9月のアンセット・オーストラリア空港の崩壊後、オーストラリアの国内市場で急速に地位を確立しました。その後はブリスベン、メルボルン、シドニーをハブとして、オーストラリア国内の32都市に直接乗り入れるまでに成長しました。 [/INST] 我、りんえもんは思う。 ヴァージン・オーストラリア航空は、2000年8月31日にヴァージン・ブルー航空として、2機の航空機で単一路線の運航を開始しましたでござる。知らんけど。


In [4]:
# 量子化パラメータ
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # 量子化の有効化
    bnb_4bit_quant_type="nf4",  # 量子化種別 (fp4 or nf4)
    bnb_4bit_compute_dtype=torch.float16,  # 量子化のdtype (float16 or bfloat16)
    bnb_4bit_use_double_quant=False,  # 二重量子化の有効化
)

model_name = "stabilityai/japanese-stablelm-base-alpha-7b"
# モデルの準備
model = AutoModelForCausalLM.from_pretrained(
    model_name,  # モデル名
    quantization_config=bnb_config,  # 量子化パラメータ
    device_map={"": 0},  # モデル全体をGPU0にロード
    trust_remote_code= True
)
model.config.use_cache = False  # キャッシュ (学習時はFalse)
model.config.pretraining_tp = 1  # 事前学習で使用したテンソル並列ランク

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

# モデルの準備
model_name = "stabilityai/japanese-stablelm-base-alpha-7b"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
# トークナイザーの準備
tokenizer = AutoTokenizer.from_pretrained(
    "novelai/nerdstash-tokenizer-v1",  # モデル名
    use_fast=False,  # Fastトークナイザーの有効化
    add_eos_token=True,  # データへのEOSの追加を指示
    trust_remote_code=True
)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.padding_side = "right" # fp16でのオーバーフロー問題対策

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=True`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [7]:
# LoRAパラメータ
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=8,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "query_key_value",
        "dense",
        "packed_input_proj",
        "out_proj",
    ]
)

# 学習パラメータ
training_arguments = TrainingArguments(
    output_dir="./content/drive/MyDrive/Workspace/QA/train_logs",  # 出力ディレクトリ
    fp16=True,  # fp16学習の有効化
    bf16=False,  # bf16学習の有効化
    max_steps=300,  # 学習ステップ数
    per_device_train_batch_size=4,  # 学習用のGPUあたりのバッチサイズ
    gradient_accumulation_steps=1,  # 勾配を蓄積するための更新ステップの数
    optim="paged_adamw_32bit",  # オプティマイザ
    learning_rate=2e-4,  # 初期学習率
    lr_scheduler_type="cosine",  # 学習率スケジュール
    max_grad_norm=0.3,  # 最大法線勾配 (勾配クリッピング)
    warmup_ratio=0.03,  # 線形ウォームアップのステップ比率 (0から学習率まで)
    weight_decay=0.001,  # bias/LayerNormウェイトを除く全レイヤーに適用するウェイト減衰
    save_steps=25,  # 何ステップ毎にチェックポイントを保存するか
    logging_steps=25,  # 何ステップ毎にログを記録するか
    group_by_length=True,  # シーケンスを同じ長さのバッチにグループ化 (メモリ節約)
    report_to="tensorboard"  # レポート
)

# SFTパラメータ
trainer = SFTTrainer(
    model=model,  # モデル
    tokenizer=tokenizer,  # トークナイザー
    train_dataset=dataset,  # データセット
    dataset_text_field="text",  # データセットのtext列
    peft_config=peft_config,  # PEFTパラメータ
    args=training_arguments,  # 学習パラメータ
    max_seq_length=None,  # 使用する最大シーケンス長
    packing=False,  # 同じ入力シーケンスに複数サンプルをパッキング(効率を高める)
)

# モデルの学習
trainer.train()
trainer.model.save_pretrained("/content/drive/MyDrive/Workspace/QA/lora_model")

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:122: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:166: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
Parameter 'function'=<function SFTTrainer._prepare_non_packed_dataloader.<locals>.tokenize at 0x7fb3903bfb50> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/15015 [00:00<?, ? examples/s]

Step,Training Loss
25,2.240200
50,1.755100
75,2.137200
100,1.624500
125,2.071500
150,1.661400
175,2.023400
200,1.516500
225,2.047400
250,1.585100


In [8]:
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer
import torch

# モデルの読み込み
model = AutoPeftModelForCausalLM.from_pretrained(
    "/content/drive/MyDrive/Workspace/QA/lora_model",
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)

# トークナイザーの準備
tokenizer = AutoTokenizer.from_pretrained(
    "novelai/nerdstash-tokenizer-v1",  # モデル名
    use_fast=False,  # Fastトークナイザーの有効化
    add_eos_token=True,  # データへのEOSの追加を指示
    trust_remote_code=True
)

tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# マージして保存
model = model.merge_and_unload()
model.save_pretrained("/content/drive/MyDrive/Workspace/QA/marged_model", safe_serialization=True)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [9]:
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer
import torch

# モデルの準備
model = AutoModelForCausalLM.from_pretrained(
    "/content/drive/MyDrive/Workspace/QA/marged_model",
    torch_dtype=torch.bfloat16,
    load_in_4bit=True,  # 4bit量子化
    device_map={"": 0},
    trust_remote_code=True
)

# トークナイザーの準備
tokenizer = AutoTokenizer.from_pretrained(
    "novelai/nerdstash-tokenizer-v1",  # モデル名
    use_fast=False,  # Fastトークナイザーの有効化
    add_eos_token=True,  # データへのEOSの追加を指示
    trust_remote_code=True
)

tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
import torch
from transformers import pipeline
from typing import Optional, List, Mapping, Any

from llama_index import (
    ServiceContext,
    SimpleDirectoryReader,
    LangchainEmbedding,
    ListIndex
)
from llama_index.callbacks import CallbackManager
from llama_index.llms import (
    CustomLLM,
    CompletionResponse,
    CompletionResponseGen,
    LLMMetadata,
)
from llama_index.llms.base import llm_completion_callback

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)
pipe.tokenizer.pad_token_id = pipe.model.config.pad_token_id
pipe.tokenizer.eos_token_id = pipe.model.config.eos_token_id

# set context window size
context_window = 2048
# set number of output tokens
num_output = 256

class OurLLM(CustomLLM):

    @property
    def metadata(self) -> LLMMetadata:
        """Get LLM metadata."""
        return LLMMetadata(
            context_window=context_window,
            num_output=num_output,
            model_name=model_name
        )

    @llm_completion_callback()
    def complete(self, prompt: str, **kwargs: Any) -> CompletionResponse:
        prompt_length = len(prompt)
        response = pipeline(prompt, max_new_tokens=num_output)[0]["generated_text"]

        # only return newly generated tokens
        text = response[prompt_length:]
        return CompletionResponse(text=text)

    @llm_completion_callback()
    def stream_complete(self, prompt: str, **kwargs: Any) -> CompletionResponseGen:
        raise NotImplementedError()

llm = OurLLM()

In [11]:
from llama_index import SimpleDirectoryReader

# ドキュメントの読み込み
documents = SimpleDirectoryReader("/content/drive/MyDrive/Workspace/QA/data").load_data()

In [12]:
from langchain.embeddings import HuggingFaceEmbeddings
from llama_index import LangchainEmbedding

# 埋め込みモデルの準備
embed_model = LangchainEmbedding(
    HuggingFaceEmbeddings(model_name="oshizo/sbert-jsnli-luke-japanese-base-lite")
)

In [13]:
from llama_index.langchain_helpers.text_splitter import TokenTextSplitter
from llama_index.node_parser import SimpleNodeParser

text_splitter = TokenTextSplitter(
    separator=" ",
    chunk_size=514,
    chunk_overlap=20,
    tokenizer=tokenizer
)

# ノードパーサーの準備
node_parser = SimpleNodeParser(text_splitter=text_splitter)

In [14]:
from llama_index import ServiceContext

# サービスコンテキストの準備
service_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model=embed_model,
    node_parser=node_parser,
)

In [15]:
from llama_index import VectorStoreIndex

# インデックスの作成
index = VectorStoreIndex.from_documents(
    documents,
    service_context=service_context,
)

In [16]:
from llama_index.prompts.prompts import QuestionAnswerPrompt

# QAテンプレートの準備
qa_template = QuestionAnswerPrompt("""<s>[INST] <<SYS>>
あなたは誠実で優秀な日本人のアシスタントです。
<</SYS>>

{query_str} [/INST]
""")

In [17]:
# クエリエンジンの作成
query_engine = index.as_query_engine(
    similarity_top_k=3,
    text_qa_template=qa_template,
)

In [18]:
response = query_engine.query( "クラウドセキュリティのガイドラインについて要約して")
print(response)

KeyError: ignored